In [1]:
from transformers import pipeline

def classify_intent(text):
    classifier = pipeline('sentiment-analysis')
    result = classifier(text)
    intent = result[0]['label']
    return intent

In [25]:
prompt = '''Suppose you are a customer support system for a bank now you have to determine the intent of the {question} provided by the user in three categories:
            1.Placing an order
            2.Account inquiries
            3.technical support
            and answer them accordingly 
        '''

In [6]:
from dotenv import load_dotenv
load_dotenv('.env.example')

True

In [20]:
from elasticsearch import Elasticsearch
import os 
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

es_pass = os.getenv('ELASTICSEARCH_KEY')
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic",es_pass),
    verify_certs=False
)
es.ping()

/Users/ripeshghimire/anaconda3/envs/sujan/lib/python3.11/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


True

In [23]:
index_name = "vector_index"  # Replace with the name of one of your indices
aliases = es.indices.get_alias(index=index_name)
print(aliases)


NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [vector_index]', vector_index, index_or_alias)

In [ ]:
from langchain_elasticsearch import ElasticsearchStore
store = ElasticsearchStore(
    index_name='vector_index',
)


In [5]:
import os

In [17]:
import google.generativeai as genai
from dotenv import load_dotenv
api_key = os.getenv('API_KEY')

In [20]:
import os 
key = os.getenv('ELASTICSEARCH_KEY')

In [21]:
print(key)

None


In [22]:

load_dotenv('.env.example')
es_pass = os.getenv('ELASTICSEARCH_KEY')


In [23]:
print(es_pass)

elasticsearch123


In [25]:
from elasticsearch import Elasticsearch

In [26]:
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=('elastic','elastisearch123'),
    verify_certs=False
)
es.ping()

False

In [34]:
from elasticsearch import Elasticsearch
import logging
import urllib3

# Disable insecure HTTPS warnings (only for development/testing)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

logging.basicConfig(level=logging.DEBUG)
es = Elasticsearch(
                "https://localhost:9200",  # Note the https here
                basic_auth=("elastic", "elasticsearch123"),
                verify_certs=False,  # For development only. In production, use proper certificates
                ssl_show_warn=False  # Suppresses SSL warnings
            )
es.ping()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): localhost:9200


DEBUG:urllib3.connectionpool:https://localhost:9200 "HEAD / HTTP/1.1" 200 0
INFO:elastic_transport.transport:HEAD https://localhost:9200/ [status:200 duration:0.133s]


True

In [32]:
from langchain_elasticsearch import ElasticsearchRetriever
from typing import Dict
from langchain_community.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
def vector_query(search_query: str) -> Dict:
    try:
        vector = embedding.embed_query(search_query)  # Generate embeddings for the query
        return {
            "knn": {
                "field": "vector",
                "query_vector": vector,
                "k": 1,
                "num_candidates": 10
            }
        }
    except KeyError as e:
        print(f"KeyError: {e}")
        return {}

In [37]:
from langchain_elasticsearch import ElasticsearchRetriever
retriever = ElasticsearchRetriever(
    es_client = es,
    index_name = "vector_index",
    body_func = vector_query,
    content_field = 'text'
)
retirve = retriever.invoke('what is this about')

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": [[12840, 374, 420, 922]], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:Resetting dropped connection: api.openai.com
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=21 request_id=req_372bc59ce54daf022950e1076e5b77bf response_code=200
DEBUG:urllib3.connectionpool:https://localhost:9200 "POST /vector_index/_search HTTP/1.1" 200 None
INFO:elastic_transport.transport:POST https://localhost:9200/vector_index/_search [status:200 duration:0.044s]


In [38]:
retirve

[Document(page_content='its fulfillment and after-sales services. This system ensures that orders are managed efficiently,\nwhich is crucial for customer satisfaction and business success. Customer Support Guide', metadata={'_index': 'vector_index', '_id': '7ae7feab-72dc-427d-8246-8859afee528f', '_score': 0.8632355, '_source': {'metadata': {}, 'vector': [-0.013021944767495876, 0.004747030571944922, -0.014018522372740714, -0.01587880112799132, -0.021765251843809064, 0.01779222953401493, -0.005627340463948278, -0.0088163879625414, 0.0008744966926058063, -0.013580028673467845, -0.0037969599588644144, 0.007726796838629211, -0.015427018386210608, -0.003680692711284243, -0.008185222239018597, -0.01676907663772228, 0.02390457068940041, -0.023997583742406448, -0.009567142961440678, -0.024329777208810684, -0.008437688900956768, 0.0008744966926058063, -0.006600664340280294, -0.011248036872303946, 0.019958124601976712, -0.0037670626400202953, 0.010258103322651717, -0.027930743581290173, 0.0115270

In [48]:
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI()
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [49]:
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_elasticsearch import ElasticsearchRetriever

def get_conversation_chain(es, index_name, vector_query):
    llm = ChatOpenAI()
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    
    retriever = ElasticsearchRetriever(
        es_client=es,
        index_name=index_name,
        body_func=vector_query,
        content_field='text'
    )
    
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )
    
    return conversation_chain


In [53]:
conversation_chain = get_conversation_chain(es, index_name='vector_index', vector_query=vector_query)
response = conversation_chain({"question": "what is this about"})
response

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/embeddings
DEBUG:openai:api_version=None data='{"input": [[12840, 374, 420, 922]], "model": "text-embedding-ada-002", "encoding_format": "base64"}' message='Post details'
DEBUG:urllib3.connectionpool:https://api.openai.com:443 "POST /v1/embeddings HTTP/1.1" 200 None
DEBUG:openai:message='OpenAI API response' path=https://api.openai.com/v1/embeddings processing_ms=19 request_id=req_2dbfa04a897e59f9b5dec49bb983f650 response_code=200
DEBUG:urllib3.connectionpool:https://localhost:9200 "POST /vector_index/_search HTTP/1.1" 200 None
INFO:elastic_transport.transport:POST https://localhost:9200/vector_index/_search [status:200 duration:0.096s]
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"messages": [{"role": "system", "content": "Use the following pieces of context to answer the user\'s question. \\nIf you don

{'question': 'what is this about',
 'chat_history': [HumanMessage(content='what is this about'),
  AIMessage(content='This is about a system that focuses on order management, fulfillment, and after-sales services to ensure customer satisfaction and business success. It appears to be a Customer Support Guide that provides information on how to effectively manage orders and provide good after-sales services.')],
 'answer': 'This is about a system that focuses on order management, fulfillment, and after-sales services to ensure customer satisfaction and business success. It appears to be a Customer Support Guide that provides information on how to effectively manage orders and provide good after-sales services.'}